In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

In [2]:
#reading in the data

routes = pd.read_csv('GTFS data waikato/routes.txt')

trips = pd.read_csv('GTFS data waikato/trips.txt')

stops = pd.read_csv('GTFS data waikato/stop_times.txt', low_memory=False)

calendar = pd.read_csv("GTFS data waikato/calendar.txt")
exceptions = pd.read_csv('GTFS data waikato/calendar_dates.txt')


In [3]:
# reducing stops table to necessary data only

stops = stops[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'shape_dist_traveled','timepoint']]

# Select only the rows with the minimum and maximum 'stop_sequence' for each unique 'trip_id'
min_max_stops = stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmin()] #minimum
min_max_stops = pd.concat([min_max_stops, stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()]]) #maximum

In [4]:
# Calculate the time difference between the first and last stop sequence for each trip_id and convert to hours
min_max_stops['trip_duration'] = min_max_stops.groupby('trip_id')['departure_time'].transform(lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).total_seconds() / 3600)

In [5]:
# calculate bus speed

min_max_stops['speed'] = min_max_stops['shape_dist_traveled']/(1000*min_max_stops['trip_duration'])

In [6]:
# Now select only entries for the last stop, remove entries for the 1st stop of the trip
min_max_stops = min_max_stops[min_max_stops['stop_sequence'] != 1]

,trip_id,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
count,7067.000000,7.067000e+03,7067.000000,7067.000000,7067.000000,7067.000000,7067.000000
mean,3593.278336,1.400436e+07,37.516061,19987.086145,0.998868,0.705931,26.603834
std,2324.656770,2.778971e+03,15.613480,18288.929848,0.033629,0.407792,6.969872
min,1.000000,1.400106e+07,4.000000,3808.110805,0.000000,0.100000,13.987626
25%,1767.500000,1.400253e+07,28.000000,10406.747951,1.000000,0.416667,22.627042
50%,3534.000000,1.400253e+07,34.000000,12975.730830,1.000000,0.533333,25.352968
75%,5300.500000,1.400799e+07,42.000000,25882.780541,1.000000,0.833333,28.080060
max,21031.000000,1.400897e+07,77.000000,135294.119197,1.000000,2.566667,65.767724


In [7]:
# covert trip_id to integer which is currently as object and therefore is not possible to do a merge
min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)

In [8]:
# Merge dataframes based on 'trip_id' column with 'outer' join
stops_trips = pd.merge(trips[['route_id', 'service_id', 'trip_id', 'trip_short_name', 'trip_headsign', 'direction_id',
                             'block_id', 'shape_id', 'operator_id']], min_max_stops, on=['trip_id'], how='outer')

In [9]:
# merge tirp, stop_times and routes data

routes_trips_stops = pd.merge(routes[['agency_id','route_id','route_short_name','route_long_name','route_type',
                                      'route_sort_order']], 
                              stops_trips[['route_id','service_id','trip_id','trip_short_name','trip_headsign', 'operator_id', 'shape_dist_traveled', 'trip_duration', 'speed']], on=['route_id'], how='outer')

In [14]:
# Manipulate calendar dataframe

# Convert start_date and end_date columns to datetime
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

# Filter for year 2022
calendar = calendar[calendar['start_date'].dt.year == 2022]

# change the end date of service to last day of 2022 to avoid having data beyond 2022

# Define the threshold date
threshold_date = pd.to_datetime('2022-12-31')

# Replace values greater than the threshold with the threshold date
calendar.loc[calendar['end_date'] > threshold_date, 'end_date'] = threshold_date


In [70]:
# The resulting dataframe will have a separate row for each route and each month when a route runs. 
#So, if the bus runs all year, it will have 12 rows for that route, with monnths 1 to 12.
new_calendar = pd.DataFrame()

for i in range(len(calendar)):
    start_dates = []

    dt_start = calendar.iloc[i,9]  #ith row of start_date
    dt_end = calendar.iloc[i,10]  #ith row of end date

    one_day = datetime.timedelta(1)
    start_dates = [dt_start]
    end_dates = []
    today = dt_start

    while today < dt_end:
        tomorrow = today + one_day
        if tomorrow.month != today.month:
            start_dates.append(tomorrow)
            end_dates.append(today)
        today = tomorrow

    end_dates.append(dt_end)

    for j in range(len(start_dates)):
        new_row = calendar.iloc[i].copy()
        new_row['start_date'] = start_dates[j]
        new_row['end_date'] = end_dates[j]
        new_row['month'] = start_dates[j].month
        new_calendar = new_calendar.append(new_row,ignore_index=True)

new_calendar.to_csv("new_calendar.csv")


C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
C:\Users\oxant\AppData\Local\Temp\ipykernel_24640\746561660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_ca

In [46]:
# Function to count number of days a bus runs between start and end date considering the weekday columns
# the resulting dataframe will have day_count column
from datetime import timedelta

def count_days_excluding(row):
    current_date = row['start_date']
    day_count = 0
    
    while current_date <= row['end_date']:
        weekday_column = current_date.strftime('%A').lower() 
        if row[weekday_column] == 1:
            day_count += 1
        current_date += timedelta(days=1)
    
    return day_count

In [71]:
# Apply the function to each row and add the result as a new column
new_calendar['day_count'] = new_calendar.apply(count_days_excluding, axis=1)

In [72]:
#check the entries for specific trip
new_calendar[new_calendar['service_id'] == 'WKO00_SSu_20220201_20220501']

,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,month,day_count
12,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-02-01,2022-02-28,2,8
13,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-03-01,2022-03-31,3,8
14,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-04-01,2022-04-30,4,9
15,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-05-01,2022-05-01,5,1


In [73]:
#the next task is to update day cout based on calendat_dates.txt which includes public holiday exclusions or additions

calendar_exceptions = pd.read_csv("GTFS data waikato/calendar_dates.txt")
calendar_exceptions['date'] = pd.to_datetime(calendar_exceptions['date'], format='%Y%m%d')

# Filter for year 2022
calendar_exceptions = calendar_exceptions[calendar_exceptions['date'].dt.year == 2022]

Comment: The following code should not be included in the final version. It is checking which service_ids are not included in calendar dataframe and therefore will not be accoutned for.

In [69]:
# Extract unique service_id values from calendar_exceptions
unique_service_ids_exceptions = calendar_exceptions['service_id'].unique()

# Find missing service_id values and their corresponding exception_type using list comprehension
missing_service_info = [(service_id, calendar_exceptions[calendar_exceptions['service_id'] == service_id]['exception_type'].values[0])
                        for service_id in unique_service_ids_exceptions
                        if service_id not in new_calendar['service_id'].values]

print("Missing service_id values and their corresponding exception_type:")
len(missing_service_info)

Missing service_id values and their corresponding exception_type:


28

In [74]:
#this code will update day_count for each service_id to incorporate public holiday exceptions

# Loop through each row in the calendar_exceptions dataframe
for index, row in calendar_exceptions.iterrows():
    service_id = row['service_id']
    date = pd.to_datetime(row['date'])  # Convert date to datetime format
    month = row['date'].month  # Extract the month component from the datetime
    
    # Find rows in calendar that match the service_id and month
    rows_to_update = new_calendar[(new_calendar['service_id'] == service_id) & (new_calendar['month'] == month)]
    
    # Update day_count based on exception_type
    if row['exception_type'] == 2:
        new_calendar.loc[rows_to_update.index, 'day_count'] -= 1
    elif row['exception_type'] == 1:
        new_calendar.loc[rows_to_update.index, 'day_count'] += 1

,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,month,day_count
12,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-02-01,2022-02-28,2,9
13,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-03-01,2022-03-31,3,8
14,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-04-01,2022-04-30,4,12
15,WKO00_SSu_20220201_20220501,00_Year_Round_Until_1st_May (Weekend),0,0,0,0,0,1,1,2022-05-01,2022-05-01,5,1


In [ ]:
# Ceck that it has updated for a specific trip
new_calendar[new_calendar['service_id'] == 'WKO00_SSu_20220201_20220501']

In [75]:
# Merge routes_trips_stors and new_calendar dataframes based on 'service_id' column with 'outer' join
trips_routes_calendar = pd.merge(new_calendar[['service_id','service_name','month','day_count']], routes_trips_stops, on=['service_id'], how='outer')

trips_routes_calendar.describe()


,month,day_count,agency_id,route_id,route_type,route_sort_order,trip_id,trip_short_name,operator_id,shape_dist_traveled,trip_duration,speed
count,26483.000000,26483.000000,30464.0,30464.000000,30464.000000,30464.000000,30462.000000,98.000000,30462.000000,30462.000000,30462.000000,30462.000000
mean,6.973492,14.940717,558.0,7664.486541,2.995207,28.411535,3233.959097,104.500000,37.897315,19657.948177,0.735110,25.968795
std,3.143026,7.588177,0.0,6388.294060,0.069063,23.082534,2309.016443,0.502571,2.527072,13951.623622,0.378355,5.911156
min,1.000000,0.000000,558.0,3771.000000,2.000000,0.000000,1.000000,104.000000,0.000000,3808.110805,0.100000,13.987626
25%,4.000000,8.000000,558.0,4606.000000,3.000000,5.750000,1548.000000,104.000000,38.000000,10519.686987,0.433333,22.569215
50%,7.000000,19.000000,558.0,4622.000000,3.000000,30.000000,2835.000000,104.500000,38.000000,14389.107809,0.616667,25.255184
75%,10.000000,22.000000,558.0,9032.000000,3.000000,43.000000,4721.000000,105.000000,38.000000,29130.674763,0.933333,27.732985
max,12.000000,23.000000,558.0,65445.000000,3.000000,74.000000,21031.000000,105.000000,44.000000,135294.119197,2.566667,65.767724


In [79]:
# investigating NAs
trips_routes_calendar.isna().sum()

service_id                 2
service_name            3981
month                   3981
day_count               3981
agency_id                  0
route_id                   0
route_short_name           0
route_long_name            0
route_type                 0
route_sort_order           0
trip_id                    2
trip_short_name        30366
trip_headsign              2
operator_id                2
shape_dist_traveled        2
trip_duration              2
speed                      2
dtype: int64